In [1]:
import pandas as pd
from global_variables.global_variables import filename_proc, sleep_activities

In [2]:
#download just the sleep data
df = pd.read_csv(filename_proc)
df = df[sleep_activities]
# print(df)

In [3]:
from supporting_func.data_summaries import strip_activity, combine_sleep

#create pd start sleep, stop sleep, time slept, date
sleep_summary = strip_activity('Sleep', df)
print(sleep_summary)


In [4]:
from global_variables.global_variables import start_date, end_date
daily_sleep_sum = combine_sleep(sleep_summary,start_date, end_date)
print(daily_sleep_sum)

          Date-nightof  total_sleep  sleep_count
0  2012-07-20 12:00:00     0.000000            0
1  2012-07-21 12:00:00     6.433333            2
2  2012-07-22 12:00:00     7.100000            3
3  2012-07-23 12:00:00     6.866667            2
4  2012-07-24 12:00:00     6.250000            1
5  2012-07-25 12:00:00     7.416667            1
6  2012-07-26 12:00:00     7.000000            1
7  2012-07-27 12:00:00     5.400000            1
8  2012-07-28 12:00:00     5.750000            1
9  2012-07-29 12:00:00     7.833333            2
10 2012-07-30 12:00:00     6.816667            1
11 2012-07-31 12:00:00     5.450000            1
12 2012-08-01 12:00:00     6.416667            1
13 2012-08-02 12:00:00     7.066667            1
14 2012-08-03 12:00:00     6.450000            1
15 2012-08-04 12:00:00     7.016667            1
16 2012-08-05 12:00:00     5.050000            1
17 2012-08-06 12:00:00     6.900000            1
18 2012-08-07 12:00:00     5.750000            1
19 2012-08-08 12:00:

In [5]:
#find the distributions for time slept, average bed time, average wake time